In [ ]:
import dspy
import os

In [ ]:
llm = dspy.LM(model='gpt-3.5-turbo', max_tokens=150)

In [32]:
import dspy
import os

# --- LLM Configuration ---
try:
    llm = dspy.LM(model='gpt-4.1-nano', max_tokens=150)
    dspy.settings.configure(lm=llm)
    print("✅ DSPy configured successfully with OpenAI.")
except Exception as e:
    print(f"❌ Failed to configure OpenAI. Please check your OPENAI_API_KEY. Error: {e}")
    exit()

# --- Custom Assertion Logic ---
class LocalAssertionError(Exception):
    pass

def local_assert(condition, message):
    if not condition:
        raise LocalAssertionError(message)

# --- Meta-Corrector with a STRATEGIC Signature ---
class SynthesizeInstruction(dspy.Signature):
    """Given a history of failed attempts and a set of required keywords, devise a
concrete, actionable strategy for a language model to follow. The goal is to generate
an output that meets all constraints. For example, if outputs are rejected for missing
keywords and then for being too long, suggest a specific technique like "Abbreviate
'generator' to 'gen' and 'discriminator' to 'discrim' to include all keywords while
staying under the character limit."
    """
    
    attempt_history = dspy.InputField(
        desc="A list of tuples, where each tuple contains a failed output and the corresponding error message."
    )
    required_keywords = dspy.InputField(desc="The keywords that must be included.")
    original_task_description = dspy.InputField(
        desc="The original high-level goal."
    )
    synthesized_instruction = dspy.OutputField(
        desc="A new, specific, and strategic instruction that resolves the conflict."
    )

# --- Meta-Corrector with the FINAL Signature ---
class SynthesizeInstruction(dspy.Signature):
    """
    Given a history of failed attempts, required keywords, and a character limit, devise a
    concrete, actionable strategy for a language model. Then, formulate a complete, new instruction
    that integrates this strategy while explicitly restating the keywords and character limit.

    For example: "Your new strategy is to use abbreviations. Create a tweet under 100 characters
    that includes the keywords 'GAN', 'generator', and 'discriminator'."
    """
    
    attempt_history = dspy.InputField(
        desc="A list of tuples, where each tuple contains a failed output and the corresponding error message."
    )
    required_keywords = dspy.InputField(desc="The keywords that must be included.")
    character_limit = dspy.InputField(desc="The maximum character limit for the output.")
    original_task_description = dspy.InputField(
        desc="The original high-level goal."
    )
    synthesized_instruction = dspy.OutputField(
        desc="A new, complete, and strategic instruction that restates all constraints."
    )



class MetaCorrector(dspy.Module):
    def __init__(self):
        super().__init__()
        self.synthesizer = dspy.Predict(SynthesizeInstruction)

    def forward(self, attempt_history, keywords, char_limit, original_task_description):
        formatted_history = "\n".join([f"- Attempted Output: '{att[0]}', Error: '{att[1]}'" for att in attempt_history])
        return self.synthesizer(
            attempt_history=formatted_history,
            required_keywords=str(keywords),
            character_limit=str(char_limit),
            original_task_description=original_task_description
        )

# --- Helper Function ---
def contains_keywords(text, keywords):
    return all(k.lower() in text.lower() for k in keywords)

# --- Signatures and Modules ---
class GenerateTweet(dspy.Signature):
    """Summarize a technical text into a tweet, following specific instructions."""
    source_text = dspy.InputField()
    feedback = dspy.InputField()
    tweet = dspy.OutputField()


class TweetSummarizer(dspy.Module):
    def __init__(self, max_attempts=5):
        super().__init__()
        self.generate_tweet = dspy.Predict(GenerateTweet)
        self.meta_corrector = MetaCorrector()
        self.max_attempts = max_attempts
        self.char_limit = 100 # Define character limit once

    def forward(self, source_text, keywords):
        failure_history = []
        feedback_instruction = "Summarize the source text into a tweet."

        for attempt in range(self.max_attempts):
            try:
                output = self.generate_tweet(
                    source_text=source_text, feedback=feedback_instruction
                )
                tweet = output.tweet
                local_assert(contains_keywords(tweet, keywords), f"Tweet must include these keywords: {keywords}")
                local_assert(len(tweet) < self.char_limit, f"Tweet must be very concise (under {self.char_limit} characters).")

                print(f"✅ Success: {tweet}")
                return tweet

            except LocalAssertionError as e:
                feedback_message = str(e)
                failure_history.append((tweet, feedback_message))
                print(f"🚨 Attempt {attempt+1} Failed: {feedback_message} (Tweet: '{tweet}')")
                
                error_messages = [err for _, err in failure_history]

                if (len(error_messages) >= 3 and error_messages[-1] == error_messages[-3] and error_messages[-1] != error_messages[-2]):
                    print("\n--- META-SELF-REFINING: PING-PONG LOOP DETECTED ---")
                    conflicting_attempts = failure_history[-3:]
                    print(f"💥 Conflicting History: {conflicting_attempts}")

                    # --- THIS IS THE FIX ---
                    # We now pass the char_limit to the meta_corrector
                    correction = self.meta_corrector(
                        attempt_history=conflicting_attempts,
                        keywords=keywords,
                        char_limit=self.char_limit, # This line was missing!
                        original_task_description=GenerateTweet.instructions
                    )
                    synthesized_instruction = correction.synthesized_instruction

                    print(f"✨ Synthesized Instruction: {synthesized_instruction}\n")

                    feedback_instruction = synthesized_instruction
                    failure_history = []
                else:
                    feedback_instruction = feedback_message
        
        print(f"\n🛑 Failed to satisfy constraints after {self.max_attempts} attempts.")
        return "Pipeline failed."

# --- Run the Pipeline ---
technical_text = """
Generative Adversarial Networks (GANs) are a class of machine learning frameworks designed by
Ian Goodfellow and his colleagues. Two neural networks, a 'generator' and a 'discriminator',
contest with each other in a zero-sum game. The generator learns to create plausible data,
while the discriminator learns to distinguish the generator's fake data from real data.
This dynamic interplay allows for the generation of highly realistic outputs.
"""
required_keywords = ["GAN", "generator", "discriminator"]

summarizer = TweetSummarizer()
result = summarizer(source_text=technical_text, keywords=required_keywords)

print(f"\n🏁 Final Result: {result}")

✅ DSPy configured successfully with OpenAI.
🚨 Attempt 1 Failed: Tweet must be very concise (under 100 characters). (Tweet: 'Generative Adversarial Networks (GANs), created by Ian Goodfellow, involve a generator and discriminator competing to produce realistic data, enabling highly convincing outputs. #AI #MachineLearning')
🚨 Attempt 2 Failed: Tweet must include these keywords: ['GAN', 'generator', 'discriminator'] (Tweet: 'GANs: two neural networks compete—one creates, the other detects fake data, enabling realistic outputs.')
🚨 Attempt 3 Failed: Tweet must be very concise (under 100 characters). (Tweet: 'Generative Adversarial Networks (GANs) involve a generator creating data and a discriminator distinguishing real from fake, leading to highly realistic outputs. #GAN #generator #discriminator')

--- META-SELF-REFINING: PING-PONG LOOP DETECTED ---
💥 Conflicting History: [('Generative Adversarial Networks (GANs), created by Ian Goodfellow, involve a generator and discriminator competing